source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://working-labs@datalakestorageaccountname.dfs.core.windows.net/bronze/geo-location/
"

In [0]:

geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en&format=json"
# geoLocationSourceAPIURL="https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

geoLocationSinkLayerName = 'bronze'
geoLocationSinkStorageAccountName = 'mydevstoragelake'
geoLocationSinkFolderName = 'geo-location'

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

In [0]:

import json
import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [0]:
dailypricingmarketnameDF=spark.sql("select distinct MARKET_NAME from pricing_analytics.gold.reporting_dim_market_gold ")
display(dailypricingmarketnameDF)

In [0]:
#retry 
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503,504])
session.mount('https://', HTTPAdapter(max_retries=retries))
marketnmes=[dailypricingMarketName["MARKET_NAME"] for dailypricingMarketName in dailypricingmarketnameDF.collect()]
geolocationAPIResponseList = []
for i, marketname in enumerate(marketnmes):
  geoLocationSourceAPIURL=f"{geoLocationSourceAPIBaseURL}{marketname}{geoLocationSourceAPIURLOptions}"
  try:
      geolocationAPIResponse = session.get(geoLocationSourceAPIURL,timeout=10)
      geolocationAPIResponse.raise_for_status()
      json_response = geolocationAPIResponse.json()
      if isinstance(json_response, dict):
          geolocationAPIResponseList.append(json_response)
  except requests.exceptions.RequestException as e:
      print(f"[Timeout] Market: {marketname}")
  except Exception as e:
      print(f"[Error] Market: {marketname}, Error: {e}")
  time.sleep(0.15)   
  if i % 100 ==0:
      print(f"Processed {i} items")
  print(i, geolocationAPIResponse.status_code, geolocationAPIResponse.text[:200])     
            
geolocationPandasDF = pd.DataFrame(geolocationAPIResponseList)
geolocationsparkDF = spark.createDataFrame(geolocationPandasDF)
(
    geolocationsparkDF
    .filter("results.admin1 IS NOT NULL")
    .write
    .mode("overwrite")
    .json(geoLocationSinkFolderPath)
)

In [0]:
geolocationbronzeDF = spark.read.json(geoLocationSinkFolderPath)
display(geolocationbronzeDF)

In [0]:
from pyspark.sql.functions import *
geolocationsilverDF = (
    geolocationbronzeDF.select(col('results.admin1').alias('stateName'),
                              col('results.admin2').alias('districtName'),
                              col('results.country').alias('countryName'),
                              col('results.latitude').alias('latitude'),
                              col('results.longitude').alias('longitude'),
                              col('results.name').alias('marketName'),
                              col('results.population').alias('population')

))
display(geolocationsilverDF)

In [0]:
display(geolocationsilverDF
        .select(explode("stateName")
        ))

In [0]:
geolocationdistrictTranDF=(geolocationsilverDF
        .select(explode("districtName").alias('districtName'),monotonically_increasing_id().alias('districtSequenceId')
))
geolocationstateTranDF=(geolocationsilverDF
        .select(explode("stateName").alias('stateName'),monotonically_increasing_id().alias('stateSequenceId')
))
geolocationcountryTranDF=(geolocationsilverDF
        .select(explode("countryName").alias('countryName'),monotonically_increasing_id().alias('countrySequenceId')
))
geolocationlatitudeTranDF=(geolocationsilverDF
        .select(explode("latitude").alias('latitude'),monotonically_increasing_id().alias('latitudeSequenceId')
))
geolocationdilongitudeTranDF=(geolocationsilverDF
        .select(explode("longitude").alias('longitude'),monotonically_increasing_id().alias('longitudeSequenceId')
))
geolocationmarketTranDF=(geolocationsilverDF
        .select(explode("marketName").alias('marketName'),monotonically_increasing_id().alias('marketSequenceId')
))
geolocationpopulationTranDF=(geolocationsilverDF
        .select(explode("population").alias('population'),monotonically_increasing_id().alias('populationSequenceId')
))       

In [0]:
geolocatiosilvertransDF= (
    geolocationstateTranDF
    .join(geolocationdistrictTranDF,col('districtSequenceId')==col('stateSequenceId'))
    .join(geolocationcountryTranDF,col('countrySequenceId')==col('stateSequenceId'))
    .join(geolocationlatitudeTranDF,col('latitudeSequenceId')==col('stateSequenceId'))
    .join(geolocationdilongitudeTranDF,col('longitudeSequenceId')==col('stateSequenceId'))
    .join(geolocationmarketTranDF,col('marketSequenceId')==col('stateSequenceId'))
    .join(geolocationpopulationTranDF,col('populationSequenceId')==col('stateSequenceId'))
    .select(col('stateName'),col('districtName'),col('countryName'),col('latitude'),col('longitude'),col('marketName'),col('population')

    )
)
display(geolocatiosilvertransDF)

In [0]:
(
    geolocatiosilvertransDF.
    write
    .mode("overwrite")
    .saveAsTable("pricing_analytics.silver.geo_location_silver")
)

In [0]:
%sql
select * from pricing_analytics.silver.geo_location_silver geoloction where geoloction.marketName='Guntur'